In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import collections
import os
import math
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [ ]:
# Data
url = 'http://mattmahoney.net/dc/'

In [ ]:
def maybe_download(file_name, expected_size):
    ''' Download the file if not present and make sure it is of expected size'''
    if not os.path.exists(file_name):
        file_name, _ = urllib.request.urlretrieve(url + file_name, file_name)
    statsinfo = os.stat(file_name)
    if statsinfo.st_size == expected_size:
        print ('Found and verified ', file_name)
    else:
        print (statsinfo.st_size)
        raise Exception('Failed to verify ' + file_name + 'check the url')
    return file_name

filename = maybe_download('text8.zip', 31344016)

In [ ]:
print (filename)
def read_data(filename):
    '''Read the first file from the zip file into a set of words'''
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data(filename)
print ('Data size: ', len(vocabulary))
print ('Vocabulary sample: ', vocabulary[:3])

In [ ]:
vocabulary_size = 5000
def build_dataset(words, num_words):
    '''Process raw words into a dataset for word2vec'''
    counts = [['UNK', -1]]
    counts.extend(collections.Counter(words).most_common(num_words - 1))
    dictionary = dict()
    for word, _ in counts:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count += 1
        data.append(index)
    counts[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, counts, dictionary, reversed_dictionary

data, count, dictionary, reversed_dictionary = build_dataset(vocabulary, vocabulary_size)

In [ ]:
# del vocabulary # Reduce memory
print ('Most common words: ', count[:5])
print ('Indices of words: ', data[:10])
print ('Sample data: ', [reversed_dictionary[i] for i in data[:10]])
print (dictionary['originated'])

In [ ]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert num_skips <= 2 * skip_window
    assert batch_size % num_skips == 0
    batch = np.ndarray(shape = (batch_size), dtype=np.int32)
    labels = np.ndarray(shape = (batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen = span)
    if data_index + span >= len(data):
        data_index = 0
    buffer.extend(data[data_index: data_index + span])
    data_index += span
    
    for i in range(batch_size//num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        if data_index == len(data):
            buffer[:] = data[:span]
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    #data_index = 0    
        #print (buffer)
    return batch, labels         

In [ ]:
print (data[0:4])
batch, labels = generate_batch(8, 2, 1)
for i in range(8):
    print (batch[i], reversed_dictionary[batch[i]], '->', labels[i, 0], reversed_dictionary[labels[i, 0]])

In [ ]:
#Build and Train a skip-gram model
batch_size = 128
embedding_size = 128
num_skips = 2
skip_window = 1

valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace = False)
print(valid_examples)
num_sampled = 64

In [ ]:
graph = tf.Graph()
with graph.as_default():
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        # print("Embeddings is ", embeddings)
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        # print ("Embed is ", embed)
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                               stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))
    
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    # print(norm)
    normalized_embeddings = embeddings / norm
    # print(normalized_embeddings)
    init = tf.global_variables_initializer()

In [ ]:
num_steps = 100001
with tf.Session(graph=graph) as sess:
    init.run()
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print ('Average loss at step:', step, ':', average_loss)
            average_loss = 0
    final_embeddings = normalized_embeddings.eval()

In [ ]:
a = tf.constant([[1.0, 2.0], [3.0, 5.0]])
with tf.Session() as sess:
    b = tf.reduce_sum(a, 1, keep_dims=True)
    print (sess.run(b))

In [ ]:
# visualize the embeddings

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert (low_dim_embs[0] >= len(labels), 'More labels than embeddings')
    plt.figure(figsize=(18, 18))
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                    xy=(x, y),
                    xytext=(5,2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
        plt.savefig(filename)
        
try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reversed_dictionary[i] for i in xrange(plot_only)]
    print (low_dim_embs[0], labels[0])
    plot_with_labels(low_dim_embs, labels)
    
except ImportError:
        print ('Install sklearn, matplotlib')